# E2B Sandbox Code Execution Tutorial

Welcome to this tutorial on **E2B (Execute to Build)**! 🦎

E2B provides secure, isolated sandbox environments for running code, especially useful for AI-generated code. This notebook will walk you through the essential concepts and show you how to use E2B for tasks like data analysis. We'll cover:

* Setting up your environment and API keys.
* Basic code execution in an E2B sandbox.
* Working with files (uploading and downloading).
* Installing custom Python packages.
* Integrating E2B with LangChain for data analysis.

## 1. Setup and API Keys

First, we need to install the necessary libraries and load our API keys from a `.env` file. This is a good practice for keeping your keys secure and out of your codebase.

In [6]:
# Uncomment to install packages
# !pip install e2b e2b-code-interpreter python-dotenv langchain langchain-community openai -q

In [7]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API keys
e2b_api_key = os.getenv("E2B_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

# Set environment variables for LangChain
os.environ["E2B_API_KEY"] = e2b_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

## 2. Basic Code Execution

Let's start by creating a sandbox and running a simple Python command.

In [9]:
from e2b_code_interpreter import Sandbox

with Sandbox() as sandbox:
    execution = sandbox.run_code('print("Hello, from the E2B sandbox!")')
    # Print the captured logs from the execution
    print(execution.logs)


Logs(stdout: ['Hello, from the E2B sandbox!\n'], stderr: [])


## 3. Working with Files

E2B makes it easy to work with files inside the sandbox. You can upload files from your local machine, and download files generated within the sandbox.

In [10]:
with Sandbox() as sandbox:
    # Create a file in the sandbox
    sandbox.filesystem.write('my_file.txt', 'This is a test file.')

    # Read the file from the sandbox
    content = sandbox.filesystem.read('my_file.txt')
    print(f"Content of my_file.txt: {content}")

    # Download the file
    # In a real scenario, you would save this to a local file
    file_bytes = sandbox.filesystem.download('my_file.txt')
    print(f"Downloaded file bytes: {file_bytes[:20]}...")

AttributeError: 'Sandbox' object has no attribute 'filesystem'

## 4. Installing Custom Packages

E2B sandboxes come with many common data science libraries pre-installed. However, you can easily install any other package you need using `pip`.

In [ ]:
with Sandbox() as sandbox:
    # Install a package
    proc = sandbox.process.start('pip install numpy')
    proc.wait()
    print(proc.stdout)
    
    # Use the installed package
    execution = sandbox.run_code('import numpy as np; print(np.random.rand(5))')
    print(f"Numpy output: {execution.stdout}")

## 5. Data Analysis with LangChain

E2B integrates seamlessly with LangChain, allowing you to create powerful data analysis agents. We'll create a simple agent that can analyze a CSV file.

In [ ]:
from langchain_community.tools.e2b_data_analysis.tool import E2BDataAnalysisTool
from langchain.agents import AgentType, initialize_agent
from langchain_openai import ChatOpenAI

# Initialize the E2B tool
e2b_tool = E2BDataAnalysisTool()

# Create a dummy CSV file
with open("dummy_data.csv", "w") as f:
    f.write("name,value\n")
    f.write("A,1\n")
    f.write("B,2\n")
    f.write("C,3\n")

# Upload the file to the E2B sandbox
with open("dummy_data.csv", "rb") as f:
    remote_path = e2b_tool.upload_file(f)

# Initialize the LangChain agent
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
agent = initialize_agent(
    [e2b_tool],
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

# Run the agent
agent.run(f"Analyze the data in {remote_path} and tell me the average value.")

## Conclusion

This tutorial covered the basics of using the E2B Sandbox for secure code execution. You've learned how to:

* Set up your environment and API keys.
* Execute basic Python code.
* Manage files within the sandbox.
* Install custom packages.
* Use E2B with LangChain for data analysis.

E2B is a powerful tool for building AI applications that can safely interact with code. For more advanced use cases, be sure to check out the official [E2B documentation](https://e2b.dev/docs).